In [1]:
import sys
sys.path.append("..")

In [2]:
from enum import Enum
from resources.utils import get_puzzle_input

### Part 1

A crop of this size requires significant logistics to transport produce, soil, fertilizer, and so on. The Elves are very busy pushing things around in carts on some kind of rudimentary system of tracks they've come up with.

Seeing as how cart-and-track systems don't appear in recorded history for another 1000 years, the Elves seem to be making this up as they go along. They haven't even figured out how to avoid collisions yet.

You map out the tracks (your puzzle input) and see where you can help.

Tracks consist of straight paths (| and -), curves (/ and \), and intersections (+). Curves connect exactly two perpendicular pieces of track; for example, this is a closed loop:

```

/----\
|    |
|    |
\----/
```

Intersections occur when two perpendicular paths cross. At an intersection, a cart is capable of turning left, turning right, or continuing straight. Here are two loops connected by two intersections:

```

/-----\
|     |
|  /--+--\
|  |  |  |
\--+--/  |
   |     |
   \-----/
```

Several carts are also on the tracks. Carts always face either up (^), down (v), left (<), or right (>). (On your initial map, the track under each cart is a straight path matching the direction the cart is facing.)

Each time a cart has the option to turn (by arriving at any intersection), it turns left the first time, goes straight the second time, turns right the third time, and then repeats those directions starting again with left the fourth time, straight the fifth time, and so on. This process is independent of the particular intersection at which the cart has arrived - that is, the cart has no per-intersection memory.

Carts all move at the same speed; they take turns moving a single step at a time. They do this based on their current location: carts on the top row move first (acting from left to right), then carts on the second row move (again from left to right), then carts on the third row, and so on. Once each cart has moved one step, the process repeats; each of these loops is called a tick.

For example, suppose there are two carts on a straight track:

```

|  |  |  |  |
v  |  |  |  |
|  v  v  |  |
|  |  |  v  X
|  |  ^  ^  |
^  ^  |  |  |
|  |  |  |  |
```

First, the top cart moves. It is facing down (v), so it moves down one square. Second, the bottom cart moves. It is facing up (^), so it moves up one square. Because all carts have moved, the first tick ends. Then, the process repeats, starting with the first cart. The first cart moves down, then the second cart moves up - right into the first cart, colliding with it! (The location of the crash is marked with an X.) This ends the second and last tick.

Here is a longer example:

```

/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   

/-->\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \->--/
  \------/   

/---v        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+>-/
  \------/   

/---\        
|   v  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+->/
  \------/   

/---\        
|   |  /----\
| /->--+-\  |
| | |  | |  |
\-+-/  \-+--^
  \------/   

/---\        
|   |  /----\
| /-+>-+-\  |
| | |  | |  ^
\-+-/  \-+--/
  \------/   

/---\        
|   |  /----\
| /-+->+-\  ^
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /----<
| /-+-->-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /---<\
| /-+--+>\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /--<-\
| /-+--+-v  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /-<--\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /<---\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-<--/
  \------/   

/---\        
|   |  v----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \<+--/
  \------/   

/---\        
|   |  /----\
| /-+--v-\  |
| | |  | |  |
\-+-/  ^-+--/
  \------/   

/---\        
|   |  /----\
| /-+--+-\  |
| | |  X |  |
\-+-/  \-+--/
  \------/
```

After following their respective paths for a while, the carts eventually crash. To help prevent crashes, you'd like to know the location of the first crash. Locations are given in X,Y coordinates, where the furthest left column is X=0 and the furthest top row is Y=0:

```

           111
 0123456789012
0/---\        
1|   |  /----\
2| /-+--+-\  |
3| | |  X |  |
4\-+-/  \-+--/
5  \------/

```

In this example, the location of the first crash is 7,3.

In [3]:
class Direction(Enum):
    LEFT = '>'
    RIGHT = '<'
    UP = '^'
    DOWN = 'v'
    NEXT = '+'
    
class Turn(Enum):
    LEFT = '>'
    RIGHT = '<'
    STRAIGHT = '-'
    
track_map = {
    '<': (Direction.LEFT, '-'),
    '>': (Direction.RIGHT, '-'),
    '^': (Direction.UP, '|'),
    'v': (Direction.DOWN, '|'),
}

move_map = {
    Direction.LEFT: (-1, 0),
    Direction.RIGHT: (1, 0),
    Direction.UP: (0, -1),
    Direction.DOWN: (0, 1),
}

turn_map = {
    (Direction.LEFT, Turn.LEFT): Direction.DOWN,
    (Direction.RIGHT, Turn.LEFT): Direction.UP,
    (Direction.UP, Turn.LEFT): Direction.LEFT,
    (Direction.DOWN, Turn.LEFT): Direction.RIGHT,
    (Direction.LEFT, Turn.RIGHT): Direction.UP,
    (Direction.RIGHT, Turn.RIGHT): Direction.DOWN,
    (Direction.UP, Turn.RIGHT): Direction.RIGHT,
    (Direction.DOWN, Turn.RIGHT): Direction.LEFT,
    (Direction.LEFT, Turn.STRAIGHT): Direction.LEFT,
    (Direction.RIGHT, Turn.STRAIGHT): Direction.RIGHT,
    (Direction.UP, Turn.STRAIGHT): Direction.UP,
    (Direction.DOWN, Turn.STRAIGHT): Direction.DOWN,
}

direction_map = {
    ('/', Direction.UP): Direction.RIGHT,
    ('/', Direction.LEFT): Direction.DOWN,
    ('/', Direction.DOWN): Direction.LEFT,
    ('/', Direction.RIGHT): Direction.UP,
    ('-', Direction.LEFT): Direction.LEFT,
    ('-', Direction.RIGHT): Direction.RIGHT,
    ('|', Direction.UP): Direction.UP,
    ('|', Direction.DOWN): Direction.DOWN,
    ('\\', Direction.DOWN): Direction.RIGHT,
    ('\\', Direction.RIGHT): Direction.DOWN,
    ('\\', Direction.LEFT): Direction.UP,
    ('\\', Direction.UP): Direction.LEFT,
    ('+', Direction.UP): Direction.NEXT,
    ('+', Direction.LEFT): Direction.NEXT,
    ('+', Direction.DOWN): Direction.NEXT,
    ('+', Direction.RIGHT): Direction.NEXT,   
}


In [4]:
class Cart:
    def __init__(self, id, x, y, direction):
        self.id = id
        self.x = x
        self.y = y
        self.direction = direction
        self.turn_choice = Turn.LEFT
        
    @property
    def position(self):
        return self.x, self.y
        
    def move(self, track):
        x_delta, y_delta = move_map[self.direction]
        self.x += x_delta
        self.y += y_delta
        track_char = track[(self.x, self.y)]
        direction = direction_map[(track_char, self.direction)]
        if direction == Direction.NEXT:
            turn = self.rotate_turn()
            self.direction = turn_map[self.direction, turn]
        else:
            self.direction = direction
            
    def rotate_turn(self):
        turn_choice = self.turn_choice
        self.turn_choice = {
            Turn.LEFT: Turn.STRAIGHT,
            Turn.STRAIGHT: Turn.RIGHT,
            Turn.RIGHT: Turn.LEFT
        }[turn_choice]
        
        return turn_choice

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, other):
        return isinstance(self, type(other)) and self.id == other.id

    def __lt__(self, other):
        return self.y < other.y or (self.y == other.y and self.x < other.x)
    
    def __repr__(self):
        return "<{}: {},{},{}>".format(
            self.id,
            self.x,
            self.y,
            self.direction,
        )

In [5]:
test_track_input = r"""/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/""".split('\n')
test_track_input

['/->-\\        ',
 '|   |  /----\\',
 '| /-+--+-\\  |',
 '| | |  | v  |',
 '\\-+-/  \\-+--/',
 '  \\------/']

In [6]:
def parse_track(lines):
    carts = []
    track = {}

    for y in range(len(lines)):
        line = lines[y]
        print
        for x in range(len(line)):
            position = x, y
            char = line[x]
            if char == ' ':
                continue
            if char in track_map:
                direction, track_char = track_map[char]
                carts.append(Cart(
                    id=len(carts),
                    x=x,
                    y=y,
                    direction=direction
                ))
            else:
                track_char = char
            
            track[position] = track_char
    
    return track, carts
                

In [7]:
test_track, carts = parse_track(test_track_input)

In [8]:
carts

[<0: 2,0,Direction.RIGHT>, <1: 9,3,Direction.DOWN>]

In [9]:
cart_1 = carts[1]
for _ in range(10):
    print(test_track[cart_1.position], cart_1)
    cart_1.move(test_track)

| <1: 9,3,Direction.DOWN>
+ <1: 9,4,Direction.RIGHT>
- <1: 10,4,Direction.RIGHT>
- <1: 11,4,Direction.RIGHT>
/ <1: 12,4,Direction.UP>
| <1: 12,3,Direction.UP>
| <1: 12,2,Direction.UP>
\ <1: 12,1,Direction.LEFT>
- <1: 11,1,Direction.LEFT>
- <1: 10,1,Direction.LEFT>


In [10]:
def solve(input, max_moves=0, tracked_cart_ids=()):
    move = 0
    track, carts = parse_track(input)

    tracked_carts = [cart for cart in carts if cart.id in tracked_cart_ids]

    print(carts)
    remaining_carts = carts
    collisions = []
    
    while True:
        if move % 10000 == 0:
            print('move: {} remaining: {}'.format(move, len(remaining_carts)))
        move += 1
        if max_moves and move > max_moves:
            return collisions, remaining_carts

        sorted_carts = sorted(remaining_carts)
        for cart in sorted_carts:
            if cart not in remaining_carts:
                continue

            if cart in tracked_carts:
                print(track[cart.position], cart)

            cart.move(track)
            
            for other_cart in remaining_carts:
                if cart != other_cart and cart.position == other_cart.position:
                    collisions.append(cart.position)
                    remaining_carts = [
                        c
                        for c in remaining_carts
                        if c not in (cart, other_cart)
                    ]
            
            if len(remaining_carts) <= 1:
                return collisions, remaining_carts
        

In [11]:
solve(test_track_input)

[<0: 2,0,Direction.RIGHT>, <1: 9,3,Direction.DOWN>]
move: 0 remaining: 2


([(7, 3)], [])

In [12]:
def get_puzzle_input(file):
    """Return the puzzle input as a list."""
    with open(file) as fh:
        lines = fh.readlines()
    return [l.rstrip() for l in lines if l.rstrip()]

In [13]:
puzzle_input = get_puzzle_input('/tmp/day_13.txt')

In [14]:
# Part 1
collisions, _ = solve(puzzle_input)
collisions[0]

[<0: 148,13,Direction.UP>, <1: 25,15,Direction.RIGHT>, <2: 47,19,Direction.RIGHT>, <3: 5,38,Direction.DOWN>, <4: 8,47,Direction.RIGHT>, <5: 118,64,Direction.RIGHT>, <6: 36,67,Direction.UP>, <7: 82,68,Direction.DOWN>, <8: 127,72,Direction.RIGHT>, <9: 133,84,Direction.RIGHT>, <10: 80,88,Direction.UP>, <11: 85,101,Direction.LEFT>, <12: 130,113,Direction.UP>, <13: 12,121,Direction.DOWN>, <14: 94,128,Direction.RIGHT>, <15: 6,132,Direction.DOWN>, <16: 64,134,Direction.LEFT>]
move: 0 remaining: 17
move: 10000 remaining: 3
move: 20000 remaining: 3


(79, 128)

In [15]:
# Tracking was useful in debugging
solve(puzzle_input, 10, (0,))

[<0: 148,13,Direction.UP>, <1: 25,15,Direction.RIGHT>, <2: 47,19,Direction.RIGHT>, <3: 5,38,Direction.DOWN>, <4: 8,47,Direction.RIGHT>, <5: 118,64,Direction.RIGHT>, <6: 36,67,Direction.UP>, <7: 82,68,Direction.DOWN>, <8: 127,72,Direction.RIGHT>, <9: 133,84,Direction.RIGHT>, <10: 80,88,Direction.UP>, <11: 85,101,Direction.LEFT>, <12: 130,113,Direction.UP>, <13: 12,121,Direction.DOWN>, <14: 94,128,Direction.RIGHT>, <15: 6,132,Direction.DOWN>, <16: 64,134,Direction.LEFT>]
move: 0 remaining: 17
| <0: 148,13,Direction.UP>
\ <0: 148,12,Direction.LEFT>
- <0: 147,12,Direction.LEFT>
+ <0: 146,12,Direction.DOWN>
| <0: 146,13,Direction.DOWN>
| <0: 146,14,Direction.DOWN>
| <0: 146,15,Direction.DOWN>
| <0: 146,16,Direction.DOWN>
| <0: 146,17,Direction.DOWN>
| <0: 146,18,Direction.DOWN>


([],
 [<0: 146,19,Direction.DOWN>,
  <1: 32,12,Direction.RIGHT>,
  <2: 53,15,Direction.RIGHT>,
  <3: 6,47,Direction.RIGHT>,
  <4: 11,40,Direction.UP>,
  <5: 121,57,Direction.UP>,
  <6: 32,61,Direction.UP>,
  <7: 87,73,Direction.RIGHT>,
  <8: 130,65,Direction.RIGHT>,
  <9: 133,76,Direction.LEFT>,
  <10: 74,84,Direction.LEFT>,
  <11: 81,107,Direction.DOWN>,
  <12: 127,106,Direction.LEFT>,
  <13: 16,127,Direction.DOWN>,
  <14: 100,124,Direction.RIGHT>,
  <15: 10,138,Direction.UP>,
  <16: 65,139,Direction.DOWN>])

### Part 2

After four very expensive crashes, a tick ends with only one cart remaining; its final location is 6,4.

What is the location of the last cart at the end of the first tick where it is the only cart left?

In [16]:
# Part 2
_, remaining = solve(puzzle_input)
remaining[0].position

[<0: 148,13,Direction.UP>, <1: 25,15,Direction.RIGHT>, <2: 47,19,Direction.RIGHT>, <3: 5,38,Direction.DOWN>, <4: 8,47,Direction.RIGHT>, <5: 118,64,Direction.RIGHT>, <6: 36,67,Direction.UP>, <7: 82,68,Direction.DOWN>, <8: 127,72,Direction.RIGHT>, <9: 133,84,Direction.RIGHT>, <10: 80,88,Direction.UP>, <11: 85,101,Direction.LEFT>, <12: 130,113,Direction.UP>, <13: 12,121,Direction.DOWN>, <14: 94,128,Direction.RIGHT>, <15: 6,132,Direction.DOWN>, <16: 64,134,Direction.LEFT>]
move: 0 remaining: 17
move: 10000 remaining: 3
move: 20000 remaining: 3


(3, 42)